In [1]:
# import packages
import requests
import pandas as pd
import numpy as np
import json
from datetime import datetime

# Extracting Data from data.gov and OneMap test

We define a test function to obtain resale price data from data.gov.sg and put it as a global dataframe known as resalepricedf.

In [14]:
def get_resale_price(weblinks, extract='n', savefile='resalepricedf.csv'):
    # data.gov.sg api server
    SERVER = 'https://data.gov.sg/api/action/datastore_search?resource_id='

    
    # limit needed if not default return is 100 records
    # there are approximately 50,000 - 350,000 maximum records per category
    LIMIT = '&limit=1000000'
    
    resalepricedf = pd.DataFrame()
    
    # for loop to pull and append data for HDB resale price information
    for resourceid in weblinks:
        url = SERVER + resourceid + LIMIT
        resp = requests.get(url)
        result = resp.json()
        result = result['result']['records']
        resalepricedf = resalepricedf.append(result)

    # we create additional columns based on the existing columns
    resalepricedf['year_sold']=resalepricedf['month'].str[:4]
    resalepricedf['month_sold']=resalepricedf['month'].str[-2:]
    
    # we convert some of the datatypes to its appriopriate types
    resalepricedf['resale_price']=resalepricedf['resale_price'].astype(float)
    resalepricedf['floor_area_sqm']=resalepricedf['floor_area_sqm'].astype(float)
    
    # we create this concatenated field to later search for postal code using OneMap
    resalepricedf['block_address']=resalepricedf['block']+' '+resalepricedf['street_name']      
        
    # When exploring the dataframe info, 
    # the earlier datasets do not have a remaining lease number
    # However since we know that all HDB leases are 99 years
    # We can impute the remaining_lease on all the values
    today = datetime.today()
    current_year = today.year
    resalepricedf['year_sold']=resalepricedf['year_sold'].astype(int)
    resalepricedf['lease_commence_date']=resalepricedf['lease_commence_date'].astype(int)  
    resalepricedf['computed_remaining_lease'] = 99 - (resalepricedf['year_sold'] - resalepricedf['lease_commence_date'])
    
    print(resalepricedf.info())
    
    if extract == 'y':
        resalepricedf.to_csv(savefile,index=False)
    else:
        pass
    
    return resalepricedf

In [3]:
# resources ID for the various HDB resale data ranges based on sale date
weblinks = [
            # 2017 onwards
            '42ff9cfe-abe5-4b54-beda-c88f9bb438ee',
            # 2015 - 2016
            '1b702208-44bf-4829-b620-4615ee19b57c',
            # 2012-2014
            '83b2fc37-ce8c-4df4-968b-370fd818138b',
            # 2000-2012
            '8c00bf08-9124-479e-aeca-7cc411d884c4',
            # 1990-1999
            'adbbddd3-30e2-445f-a123-29bee150a6fe'
            ] 

# We want to extract the raw data hence we put extract as 'y'
resalepricedf = get_resale_price(weblinks, extract='y',savefile='resalepricedf.csv')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 838477 entries, 0 to 287199
Data columns (total 16 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   town                      838477 non-null  object 
 1   flat_type                 838477 non-null  object 
 2   flat_model                838477 non-null  object 
 3   floor_area_sqm            838477 non-null  float64
 4   street_name               838477 non-null  object 
 5   resale_price              838477 non-null  float64
 6   month                     838477 non-null  object 
 7   remaining_lease           129423 non-null  object 
 8   lease_commence_date       838477 non-null  int32  
 9   storey_range              838477 non-null  object 
 10  _id                       838477 non-null  int64  
 11  block                     838477 non-null  object 
 12  year_sold                 838477 non-null  int32  
 13  month_sold                838477 non-null  o

In [4]:
resalepricedf.head()

,town,flat_type,flat_model,floor_area_sqm,street_name,resale_price,month,remaining_lease,lease_commence_date,storey_range,_id,block,year_sold,month_sold,block_address,computed_remaining_lease
0,ANG MO KIO,2 ROOM,Improved,44.0,ANG MO KIO AVE 10,232000.0,2017-01,61 years 04 months,1979,10 TO 12,1,406,2017,01,406 ANG MO KIO AVE 10,61
1,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 4,250000.0,2017-01,60 years 07 months,1978,01 TO 03,2,108,2017,01,108 ANG MO KIO AVE 4,60
2,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 5,262000.0,2017-01,62 years 05 months,1980,01 TO 03,3,602,2017,01,602 ANG MO KIO AVE 5,62
3,ANG MO KIO,3 ROOM,New Generation,68.0,ANG MO KIO AVE 10,265000.0,2017-01,62 years 01 month,1980,04 TO 06,4,465,2017,01,465 ANG MO KIO AVE 10,62
4,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 5,265000.0,2017-01,62 years 05 months,1980,01 TO 03,5,601,2017,01,601 ANG MO KIO AVE 5,62


In [5]:
# Create a array that displays all the unique addresses to lookup
addresses = resalepricedf['block_address'].unique()

In [6]:
# To get a feel of how many addresses there are
len(addresses)

9359

In [43]:
# Returns OneMap data based on block_address field and a list of addresses with errors

def get_postal_code(addresslist, extract='n', savefile='onemapresults.csv'):
    # Onemap API server
    SERVER = 'https://developers.onemap.sg/commonapi/search?searchVal='

    # Required variables for API call
    VARIABLES = '&returnGeom=Y&getAddrDetails=Y'

    onemap_results = pd.DataFrame()
    errors = []
    
    # For loop to pull and append detailed address data including postal code based on HDB street address
    for address in addresslist:
        try:
            url = SERVER + address + VARIABLES
            resp = requests.get(url)
            result = resp.json()
            result = result['results']
            result[0]['SEARCH']=address
            onemap_results = onemap_results.append(result)
        except:
            errors.append(address)
            print('Error with ' + address)
    
    # Some OneMap results have multiple entries so we need to fill in the blanks as well
    onemap_results = onemap_results.fillna(method='ffill')
    
    if extract == 'y':
        onemap_results.to_csv(savefile)
    else:
        pass
    
    # Checksum to make sure all values have been put through the API
    if len(addresslist)-len(errors) == 0:
        print('Dataframe is empty')
    elif len(addresslist)-len(onemap_results.SEARCH.unique())-len(errors) == 0:
        print('All values have been put through the API')
    else:
        print('Not all values have been put through the API')
    
    return onemap_results, errors

In [24]:
# We want to extract the raw data hence we put extract as 'y'
onemap_results, address_errors = get_postal_code(addresses,extract='y',savefile='onemapresults.csv')

Error with 3 ST. GEORGE'S RD
Error with 21 ST. GEORGE'S RD
Error with 11 ST. GEORGE'S RD
Error with 8 ST. GEORGE'S LANE
Error with 18 ST. GEORGE'S RD
Error with 15 ST. GEORGE'S RD
Error with 9 ST. GEORGE'S RD
Error with 4B ST. GEORGE'S LANE
Error with 7 ST. GEORGE'S LANE
Error with 5 ST. GEORGE'S LANE
Error with 22 ST. GEORGE'S RD
Error with 20 ST. GEORGE'S RD
Error with 13 ST. GEORGE'S RD
Error with 6 ST. GEORGE'S LANE
Error with 2 ST. GEORGE'S RD
Error with 14 ST. GEORGE'S RD
Error with 23 ST. GEORGE'S RD
Error with 16 ST. GEORGE'S RD
Error with 1 ST. GEORGE'S RD
Error with 10 ST. GEORGE'S RD
Error with 17 ST. GEORGE'S RD
Error with 19 ST. GEORGE'S RD
Error with 1A WOODLANDS CTR RD
Error with 2A WOODLANDS CTR RD
Error with 248 ANG MO KIO AVE 2
Error with 33 TAMAN HO SWEE
Error with 88 ZION RD
Error with 29 HAVELOCK RD
Error with 3 ROCHOR RD
Error with 59 SIMS DR
Error with 403 CLEMENTI AVE 1
Error with 402 CLEMENTI AVE 1
Error with 401 CLEMENTI AVE 1
Error with 313 CLEMENTI AVE 4
Err

There are bound to be some errors when searching through the address list, hence we would need to troubleshoot these manually. One common error is that the API just fails after a certain number of runs. Hence we try to run the results one more time for good measure.

In [44]:
onemap_results2, address_errors2 = get_postal_code(address_errors,extract='y',savefile='onemapresults2.csv')

Error with 3 ST. GEORGE'S RD
Error with 21 ST. GEORGE'S RD
Error with 11 ST. GEORGE'S RD
Error with 8 ST. GEORGE'S LANE
Error with 18 ST. GEORGE'S RD
Error with 15 ST. GEORGE'S RD
Error with 9 ST. GEORGE'S RD
Error with 4B ST. GEORGE'S LANE
Error with 7 ST. GEORGE'S LANE
Error with 5 ST. GEORGE'S LANE
Error with 22 ST. GEORGE'S RD
Error with 20 ST. GEORGE'S RD
Error with 13 ST. GEORGE'S RD
Error with 6 ST. GEORGE'S LANE
Error with 2 ST. GEORGE'S RD
Error with 14 ST. GEORGE'S RD
Error with 23 ST. GEORGE'S RD
Error with 16 ST. GEORGE'S RD
Error with 1 ST. GEORGE'S RD
Error with 10 ST. GEORGE'S RD
Error with 17 ST. GEORGE'S RD
Error with 19 ST. GEORGE'S RD
Error with 1A WOODLANDS CTR RD
Error with 2A WOODLANDS CTR RD
Error with 248 ANG MO KIO AVE 2
Error with 33 TAMAN HO SWEE
Error with 88 ZION RD
Error with 29 HAVELOCK RD
Error with 3 ROCHOR RD
Error with 59 SIMS DR
Error with 403 CLEMENTI AVE 1
Error with 402 CLEMENTI AVE 1
Error with 401 CLEMENTI AVE 1
Error with 313 CLEMENTI AVE 4
Err

In [ ]:
# A quick search online shows that these flats are en-bloc and hence can be ignored
# https://remembersingapore.org/singapore-en-bloc-flats/

In [51]:
# Replace St. George with St George to get OneMap to trigger the postal code
address_errors2 = [address.replace('ST.', 'ST') for address in address_errors]

In [52]:
onemap_results3, address_errors3 = get_postal_code(address_errors2,extract='y',savefile='onemapresults3.csv')

Error with 1A WOODLANDS CTR RD
Error with 2A WOODLANDS CTR RD
Error with 248 ANG MO KIO AVE 2
Error with 33 TAMAN HO SWEE
Error with 88 ZION RD
Error with 29 HAVELOCK RD
Error with 3 ROCHOR RD
Error with 59 SIMS DR
Error with 403 CLEMENTI AVE 1
Error with 402 CLEMENTI AVE 1
Error with 401 CLEMENTI AVE 1
Error with 313 CLEMENTI AVE 4
Error with 1 JLN PASAR BARU
Error with 7 TEBAN GDNS RD
Error with 168 BOON LAY DR
Error with 169 BOON LAY DR
Error with 172 BOON LAY DR
Error with 10 YUNG KUANG RD
Error with 220 BOON LAY AVE
Error with 6 UPP BOON KENG RD
Error with 30 LOR 5 TOA PAYOH
Error with 10 GHIM MOH RD
Error with 172 STIRLING RD
Error with 76 C'WEALTH DR
Error with 51 TANGLIN HALT RD
Error with 7 SELETAR WEST FARMWAY 6
Error with 83 BEDOK NTH RD
Error with 247 ANG MO KIO AVE 3
Error with 407 CLEMENTI AVE 1
Error with 409 CLEMENTI AVE 1
Error with 114 BT MERAH VIEW
Error with 89 ZION RD
Error with 111 BT MERAH VIEW
Error with 4 ROCHOR RD
Error with 62 SIMS DR
Error with 10 TEBAN GDNS

In [56]:
# We combine the new data from onemap with the existing data
onemap_results = pd.concat([onemap_results,onemap_results3])

# Checking postal code result values from OneMap

There are bound to be some errors due to OneMap's API, hence we have to check if the postal codes make sense and troubleshoot any errors manually

In [205]:
onemap_results = pd.read_csv(r'C:\Users\nicho\Desktop\Housing\onemapresults_combined.csv',index_col=0)

In [206]:
test = onemap_results.copy()

There were some postal codes that are returning a NIL value. This is happening because some of the API returns more than 1 value and the first value returned might be NIL and the second value accurate. Hence there might not be an issue if there are multiple addresses.

e.g index 0 shows NIL postal code, same address with index 1 shows a valid postal code

Hence we do a quick sense check that search values with NIL have multiple values and proceed to delete all NIL values that have multiple values and check that the number of unique addresses still remain

In [207]:
niltest=test[test.POSTAL!='NIL']
value = len(test.SEARCH.unique()) - len(niltest.SEARCH.unique())
print('There are '+str(value)+' values without an actual postal code')

There are 3 values without an actual postal code


In [208]:
# Checking which 3 addresses need manual input
checks = test[test.POSTAL=='NIL']
confirmed = test[test.POSTAL!='NIL']
manual_input=[]
for i in checks.SEARCH:
    if i in confirmed.SEARCH.tolist():
        pass
    else:
        print(i)
        manual_input.append(i)

216 CHOA CHU KANG CTRL
215 CHOA CHU KANG CTRL
11 GHIM MOH RD
11 GHIM MOH RD
11 GHIM MOH RD


In [209]:
# Manually imputing the postal code values for the 3 addresses
test.POSTAL[test.SEARCH == '216 CHOA CHU KANG CTRL'] = '680216'
test.BLK_NO[test.SEARCH == '216 CHOA CHU KANG CTRL'] = '216'
test.POSTAL[test.SEARCH == '215 CHOA CHU KANG CTRL'] = '680215' 
test.BLK_NO[test.SEARCH == '215 CHOA CHU KANG CTRL'] = '215' 
test.POSTAL[test.SEARCH == '11 GHIM MOH RD'] = '270011'
test.BLK_NO[test.SEARCH == '11 GHIM MOH RD'] = '11' 

<ipython-input-209-34fac71728c9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.POSTAL[test.SEARCH == '216 CHOA CHU KANG CTRL'] = '680216'
<ipython-input-209-34fac71728c9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.BLK_NO[test.SEARCH == '216 CHOA CHU KANG CTRL'] = '216'
<ipython-input-209-34fac71728c9>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.POSTAL[test.SEARCH == '215 CHOA CHU KANG CTRL'] = '680215'
<ipy

In [210]:
print('There are '+str(len(test.SEARCH.unique()))+' unique addresses within the dataframe')

There are 9248 unique addresses within the dataframe


In [211]:
# First we remove all the block numbers with alphabets 
test['BLK_NO'] = test['BLK_NO'].str.replace('A', '')
test['BLK_NO'] = test['BLK_NO'].str.replace('B', '')
test['BLK_NO'] = test['BLK_NO'].str.replace('C', '')
test['BLK_NO'] = test['BLK_NO'].str.replace('D', '')
test['BLK_NO'] = test['BLK_NO'].str.replace('E', '')
test['BLK_NO'] = test['BLK_NO'].str.replace('F', '')
test['BLK_NO'] = test['BLK_NO'].str.replace('G', '')
test['BLK_NO'] = test['BLK_NO'].str.replace('H', '')
test['BLK_NO'] = test['BLK_NO'].str.replace('I', '')
test['BLK_NO'] = test['BLK_NO'].str.replace('J', '')
test['BLK_NO'] = test['BLK_NO'].str.replace('K', '')
test['BLK_NO'] = test['BLK_NO'].str.replace('L', '')
test['BLK_NO'] = test['BLK_NO'].str.replace('M', '')
test['BLK_NO'] = test['BLK_NO'].str.replace('N', '')

In [212]:
# We remove all postal codes with NIL values since we know there are legitimate values within the dataset
test=test[test.POSTAL!='NIL']

In [213]:
# Extract the last 3 digits of the postal code and do a sense check against block number
test['3digit'] = test['POSTAL'].astype(str).str[-3:]
test['3digit'] = test['3digit'].astype(int)
test['BLK_NO'] = test['BLK_NO'].astype(int)
explore_issues = test[test['3digit'] != test['BLK_NO']]

In [214]:
# Visually inspect the results
explore_issues.info()
print(explore_issues)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 91 entries, 2 to 1
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   SEARCHVAL   91 non-null     object 
 1   BLK_NO      91 non-null     int32  
 2   ROAD_NAME   91 non-null     object 
 3   BUILDING    91 non-null     object 
 4   ADDRESS     91 non-null     object 
 5   POSTAL      91 non-null     object 
 6   X           91 non-null     float64
 7   Y           91 non-null     float64
 8   LATITUDE    91 non-null     float64
 9   LONGITUDE   91 non-null     float64
 10  LONGTITUDE  91 non-null     float64
 11  SEARCH      91 non-null     object 
 12  3digit      91 non-null     int32  
dtypes: float64(5), int32(2), object(6)
memory usage: 9.2+ KB
                                 SEARCHVAL  BLK_NO            ROAD_NAME  \
2     18 BEDOK SOUTH ROAD SINGAPORE 469276      18     BEDOK SOUTH ROAD   
0                      THE PINNACLE@DUXTON       1      CANTONMENT ROAD

In [215]:
# Some of the remaining errors are because block numbers with 1 digit don't start end with 00X
# Extract the last digit of the postal code and do a sense check against block number
explore_issues['1digit'] = explore_issues['POSTAL'].astype(str).str[-1:]
explore_issues['1digit'] = explore_issues['1digit'].astype(int)
explore_issues = explore_issues[explore_issues['1digit'] != explore_issues['BLK_NO']]

<ipython-input-215-6a07c36380fd>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  explore_issues['1digit'] = explore_issues['POSTAL'].astype(str).str[-1:]
<ipython-input-215-6a07c36380fd>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  explore_issues['1digit'] = explore_issues['1digit'].astype(int)


In [216]:
# Visually inspect the results

# For index values that are not 0, we will take the index 0 postal which would be correct
# Hence we need to see if they have existing 0 index values that are not NIL postal code

checks = explore_issues[explore_issues.index!=0]
confirmed = test[test.POSTAL!='NIL']
manual_checks=[]
for i in checks.SEARCH:
    if i in confirmed.SEARCH.tolist():
        pass
    else:
        print(i)
        manual_checks.append(i)

In [217]:
# For index values that are 0, we will need to manually amend any errors
explore_issues[explore_issues.index==0]

,SEARCHVAL,BLK_NO,ROAD_NAME,BUILDING,ADDRESS,POSTAL,X,Y,LATITUDE,LONGITUDE,LONGTITUDE,SEARCH,3digit,1digit
0,6 JALAN JAMBU BATU SINGAPORE 588725,6,JALAN JAMBU BATU,NIL,6 JALAN JAMBU BATU SINGAPORE 588725,588725,22944.82270,35355.45693,1.336016,103.787895,103.787895,6 JLN BATU,725,5
0,34 UPPER CROSS STREET SINGAPORE 058340,34,UPPER CROSS STREET,NIL,34 UPPER CROSS STREET SINGAPORE 058340,58340,29300.24475,29611.03593,1.284066,103.845002,103.845002,34 UPP CROSS ST,340,0
0,BEACH CENTRE,15,BEACH ROAD,BEACH CENTRE,15 BEACH ROAD BEACH CENTRE SINGAPORE 189677,189677,30468.08905,30908.06952,1.295796,103.855495,103.855495,15 BEACH RD,677,7
0,DBS BUZZ CHOA CHU KANG CENTRAL 230,6,POTONG PASIR AVENUE 2,DBS BUZZ CHOA CHU KANG CENTRAL 230,6 POTONG PASIR AVENUE 2 DBS BUZZ CHOA CHU KANG...,358361,31765.27129,34981.41899,1.332634,103.867152,103.867152,230 CHOA CHU KANG CTRL,361,1
0,20 BEDOK SOUTH ROAD SINGAPORE 469277,20,BEDOK SOUTH ROAD,NIL,20 BEDOK SOUTH ROAD SINGAPORE 469277,469277,38967.06283,33379.22634,1.318142,103.931863,103.931863,20 BEDOK STH RD,277,7
0,2 QUEEN'S ROAD SINGAPORE 266733,2,QUEEN'S ROAD,NIL,2 QUEEN'S ROAD SINGAPORE 266733,266733,25518.77925,33894.12691,1.322801,103.811024,103.811024,2 QUEEN'S RD,733,3
0,BENDEMEER INDUSTRIAL ESTATE,48,BENDEMEER ROAD,BENDEMEER INDUSTRIAL ESTATE,48 BENDEMEER ROAD BENDEMEER INDUSTRIAL ESTATE ...,339932,31136.59267,33038.95854,1.315067,103.861502,103.861502,48 BENDEMEER RD,932,2
0,BENDEMEER INDUSTRIAL ESTATE,46,BENDEMEER ROAD,BENDEMEER INDUSTRIAL ESTATE,46 BENDEMEER ROAD BENDEMEER INDUSTRIAL ESTATE ...,339931,31129.06048,33029.48367,1.314981,103.861435,103.861435,46 BENDEMEER RD,931,1
0,3 QUEEN'S ROAD SINGAPORE 266734,3,QUEEN'S ROAD,NIL,3 QUEEN'S ROAD SINGAPORE 266734,266734,25588.24676,33902.61311,1.322878,103.811648,103.811648,3 QUEEN'S RD,734,4
0,HOLLAND GROVE PARK,16,HOLLAND GROVE DRIVE,HOLLAND GROVE PARK,16 HOLLAND GROVE DRIVE HOLLAND GROVE PARK SING...,278863,22311.41801,32909.11498,1.313892,103.782204,103.782204,16 HOLLAND DR,863,3


In [218]:
test.POSTAL[test.SEARCH == '34 UPP CROSS ST'] = '050034'
test.POSTAL[test.SEARCH == '6 JLN BATU'] = '431006'
test.POSTAL[test.SEARCH == '15 BEACH RD'] = '190015'
test.POSTAL[test.SEARCH == '230 CHOA CHU KANG CTRL'] = '680230'
test.POSTAL[test.SEARCH == '20 BEDOK STH RD'] = '460020'
test.POSTAL[test.SEARCH == '2 QUEEN\'S RD'] = '260002'
test.POSTAL[test.SEARCH == '3 QUEEN\'S RD'] = '260003'
test.POSTAL[test.SEARCH == '48 BENDEMEER RD'] = '330048'
test.POSTAL[test.SEARCH == '46 BENDEMEER RD'] = '330046'
test.POSTAL[test.SEARCH == '16 HOLLAND DR'] = '271016'
test.POSTAL[test.SEARCH == '17 HOLLAND DR'] = '271017'
test.POSTAL[test.SEARCH == '15 HOLLAND DR'] = '271015'
test.POSTAL[test.SEARCH == '19 HOLLAND DR'] = '271019'
test.POSTAL[test.SEARCH == '1 ROCHOR RD'] = '180001'
test.POSTAL[test.SEARCH == '2 ROCHOR RD'] = '180002'

<ipython-input-218-22792bb70d0b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.POSTAL[test.SEARCH == '34 UPP CROSS ST'] = '050034'
<ipython-input-218-22792bb70d0b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.POSTAL[test.SEARCH == '6 JALAN BATU'] = '431006'
<ipython-input-218-22792bb70d0b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.POSTAL[test.SEARCH == '15 BEACH RD'] = '190015'
<ipython-input-218-22792bb70d

In [219]:
# We will remove the data for en-bloc units
test = test[test.SEARCH != '5 SEMBAWANG RD']
test = test[test.SEARCH != '16 OUTRAM HILL']

In [220]:
# Double checking we didn't remove more addresses than we needed to
# Number should tie to number of en-bloc units we remove in previous cell
len(onemap_results.SEARCH.unique())-len(test.SEARCH.unique())

2

In [221]:
# Syncing test checks with onemap_results
onemap_results.POSTAL[onemap_results.SEARCH == '216 CHOA CHU KANG CTRL'] = '680216'
onemap_results.BLK_NO[onemap_results.SEARCH == '216 CHOA CHU KANG CTRL'] = '216'
onemap_results.POSTAL[onemap_results.SEARCH == '215 CHOA CHU KANG CTRL'] = '680215' 
onemap_results.BLK_NO[onemap_results.SEARCH == '215 CHOA CHU KANG CTRL'] = '215' 
onemap_results.POSTAL[onemap_results.SEARCH == '11 GHIM MOH RD'] = '270011'
onemap_results.BLK_NO[onemap_results.SEARCH == '11 GHIM MOH RD'] = '11' 

onemap_results.POSTAL[onemap_results.SEARCH == '34 UPP CROSS ST'] = '050034'
onemap_results.POSTAL[onemap_results.SEARCH == '6 JLN BATU'] = '431006'
onemap_results.POSTAL[onemap_results.SEARCH == '15 BEACH RD'] = '190015'
onemap_results.POSTAL[onemap_results.SEARCH == '230 CHOA CHU KANG CTRL'] = '680230'
onemap_results.POSTAL[onemap_results.SEARCH == '20 BEDOK STH RD'] = '460020'
onemap_results.POSTAL[onemap_results.SEARCH == '2 QUEEN\'S RD'] = '260002'
onemap_results.POSTAL[onemap_results.SEARCH == '3 QUEEN\'S RD'] = '260003'
onemap_results.POSTAL[onemap_results.SEARCH == '48 BENDEMEER RD'] = '330048'
onemap_results.POSTAL[onemap_results.SEARCH == '46 BENDEMEER RD'] = '330046'
onemap_results.POSTAL[onemap_results.SEARCH == '16 HOLLAND DR'] = '271016'
onemap_results.POSTAL[onemap_results.SEARCH == '17 HOLLAND DR'] = '271017'
onemap_results.POSTAL[onemap_results.SEARCH == '15 HOLLAND DR'] = '271015'
onemap_results.POSTAL[onemap_results.SEARCH == '19 HOLLAND DR'] = '271019'
onemap_results.POSTAL[onemap_results.SEARCH == '1 ROCHOR RD'] = '180001'
onemap_results.POSTAL[onemap_results.SEARCH == '2 ROCHOR RD'] = '180002'

onemap_results = onemap_results[onemap_results.SEARCH != '5 SEMBAWANG RD']
onemap_results = onemap_results[onemap_results.SEARCH != '16 OUTRAM HILL']

onemap_results = onemap_results[onemap_results.POSTAL != 'NIL']

<ipython-input-221-1de2f2115eec>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  onemap_results.POSTAL[onemap_results.SEARCH == '216 CHOA CHU KANG CTRL'] = '680216'
<ipython-input-221-1de2f2115eec>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  onemap_results.BLK_NO[onemap_results.SEARCH == '216 CHOA CHU KANG CTRL'] = '216'
<ipython-input-221-1de2f2115eec>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  onemap_results.POSTAL[onemap

In [222]:
# Double checking we didn't remove more addresses than we needed to
print(len(onemap_results.SEARCH.unique())-len(test.SEARCH.unique())) # Number should be 0

print('There are '+str(len(onemap_results.SEARCH.unique()))+' unique addresses within the dataframe')
print('There are '+ str(len(onemap_results))+' rows in the dataframe')

0
There are 9246 unique addresses within the dataframe
There are 11462 rows in the dataframe


In [223]:
# Now we need to remove duplicate SEARCH values
mask = onemap_results.SEARCH.duplicated(keep='first')
onemap_results = onemap_results[~mask]

In [224]:
print('There are '+str(len(onemap_results.SEARCH.unique()))+' unique addresses within the dataframe')
print('There are '+ str(len(onemap_results))+' rows in the dataframe')

There are 9246 unique addresses within the dataframe
There are 9246 rows in the dataframe


In [225]:
onemap_results.to_csv('onemap_results_final.csv')


# Merging onemap_results with resalepricedf

In [226]:
to_merge = onemap_results[['SEARCH','POSTAL']]
to_merge.columns = ['block_address','postal']
to_merge.block_address = to_merge.block_address.str.replace('ST G','ST. G')

C:\Users\nicho\anaconda3\lib\site-packages\pandas\core\generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [227]:
resalepricedf = pd.read_csv(r'C:\Users\nicho\Desktop\Housing\resalepricedf.csv',index_col=0)

C:\Users\nicho\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [228]:
final_df = resalepricedf.merge(to_merge,how='left',on='block_address')
print(len(resalepricedf)-len(final_df)) #value should be 0

0


In [229]:
# We remove the initial en bloc addresses that did not get returns from OneMap
final_df.dropna(subset=['postal'])

,flat_type,flat_model,floor_area_sqm,street_name,resale_price,month,remaining_lease,lease_commence_date,storey_range,_id,block,year_sold,month_sold,block_address,computed_remaining_lease,postal
0,2 ROOM,Improved,44.0,ANG MO KIO AVE 10,232000.0,2017-01,61 years 04 months,1979,10 TO 12,1,406,2017,1,406 ANG MO KIO AVE 10,61,560406
1,3 ROOM,New Generation,67.0,ANG MO KIO AVE 4,250000.0,2017-01,60 years 07 months,1978,01 TO 03,2,108,2017,1,108 ANG MO KIO AVE 4,60,560108
2,3 ROOM,New Generation,67.0,ANG MO KIO AVE 5,262000.0,2017-01,62 years 05 months,1980,01 TO 03,3,602,2017,1,602 ANG MO KIO AVE 5,62,560602
3,3 ROOM,New Generation,68.0,ANG MO KIO AVE 10,265000.0,2017-01,62 years 01 month,1980,04 TO 06,4,465,2017,1,465 ANG MO KIO AVE 10,62,560465
4,3 ROOM,New Generation,67.0,ANG MO KIO AVE 5,265000.0,2017-01,62 years 05 months,1980,01 TO 03,5,601,2017,1,601 ANG MO KIO AVE 5,62,560601
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
838472,EXECUTIVE,APARTMENT,142.0,YISHUN ST 61,456000.0,1999-12,NaN,1987,10 TO 12,287196,611,1999,12,611 YISHUN ST 61,87,760611
838473,EXECUTIVE,APARTMENT,142.0,YISHUN CTRL,408000.0,1999-12,NaN,1988,01 TO 03,287197,324,1999,12,324 YISHUN CTRL,88,760324
838474,EXECUTIVE,MAISONETTE,146.0,YISHUN AVE 6,469000.0,1999-12,NaN,1988,07 TO 09,287198,392,1999,12,392 YISHUN AVE 6,88,760392
838475,EXECUTIVE,MAISONETTE,146.0,YISHUN RING RD,440000.0,1999-12,NaN,1988,04 TO 06,287199,356,1999,12,356 YISHUN RING RD,88,760356


In [230]:
final_df.to_csv('final_df.csv')